# Python DS-ao-Dev

## Bussiness Problem



**Star Jeans Company**

- Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendimento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda dos USA como um modelo de negócio do tipo E-commerce.

- A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação baixo e escalar a medida que forem conseguindo clientes.

- Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e o material para a fabricação de cada peça.

- Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes perguntas: ]
    1. Qual o melhor preço de venda para as calças? 
    2. Quantos tipos de calças e suas cores para o produto inicial? 
    3. Quais as matérias-prima necessárias para confeccionar as calças?
    
- As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

## Solution Planning (Input-Output-Tasks)





**Bussiness Question**

- Which is the best price for jeans?

1. Input:
    1. Fonte de dados
        - Site da H&M: https://www2.hm.com/en_us/men/products/jeans.html
        - Site da Macys: https://www.macys.com/shop/mens-clothing/mens-jeans
    2. Ferramentas
        - Python 3.8.0
        - Bibliotecas de Webscrapping ( BS4, Selenium )
        - PyCharm
        - Jupyter Notebook ( Analise e prototipagens )
        - Crontjob, Airflow
        - Streamlit
    
2. Output:
    1. A resposta para a pergunta.
        - Mediana dos preços dos concorrents.
    2. Formato da entrega
        - Tabela ou gráfico
    3. Local da entrega
        - App Streamlit
    
3. Tasks:
    1. Passo a passso para construir o cálculo da mediana ou média
        - Realizar o calculo da mediana sobre o produto, tipo e cor
    2. Definir o formato da entrega ( Visualização, Tabela, Frase )
        - Gráfico de barras com a mediana dos preço dos produtos, por tipo e cor dos últimos 30 dia
        - Tabela com as seguintes colunas: id | product_name | product_type | product_color | produ
        - Definição do schema: Colunas e seu tipo
        - Definição a infraestrutura de armazenamento ( SQLITE3 )
        - Design do ETL ( Scripts de Extração, Transformação e Carga )
        - Planejamento de Agendamento dos scripts ( dependencias entre os scripts )
        - Fazer as visualizações
        - Entrega do produto final
    3. Decidir o local de entrega ( PowerBi, Telegram, Email, Streamlit, Intranet ),
        - App com Streamlit


## Bussiness Models



“Como você planeja ganhar dinheiro”, Michael Lewis

“Um modelo de negócio descreve a lógica de criação, entrega e captura de valor por
parte de uma organização”, Alexander Osterwalder

- E-commerce:
    1. Faturamento: Vendas de um produto.
    2. Exemplo: Lojas Riachuelo, Submarino, Magazine Luiza, etc
        
- Software AS a Service ( SaaS ):
    1. Faturamento: Assinatura mensal/anual de utilização ou por usuário.
    2. Exemplo: Looker, Asana, Gmail, Salesforce.
    
- Serviço:
    1. Faturamento: Prestação de serviço por tempo ou projeto.
    2. Exemplo: Sul América, Porto Seguro, Mapfre.
    
- Mobile App:
    1. Faturamento: Venda de upgrades.
    2. Exemplo: Wildlife, Ubisoft, Games Mobile.
    
- Media Site:
    1. Faturamento: Cobrança por clicks ou visualizações de um determinado anúncio.
    2. Exemplo: Facebook, Google, UOL, G1, etc.
    
- Marketplace:
    1. Faturamento: Taxa sobre a transação entre o passageiro e o motorista.
    2. Exemplo: Uber, Ifood, 99, Elo7, Submarino.


## E-commerce Metrics

- **Growth Metrics**:
    1. Porcentagem do Marketshare
    2. Número de Clientes Novos
- **Revenue Metrics**:
    1. Número de Vendas
    2. Ticket Médio
    3. LTV ( Long Time Value )
    4. Recência Média
    5. Basket Size Médio
    6. Markup médio
- **Cost Metrics**:
    1. CAC ( Custo de aquisição de Clientes )
    2. Desconto médio
    3. Custo de Produção
    4. Taxa de devolução
    5. Custos Fixos ( Folha de pagamento, escritório, softwares )
    6. Impostos

# Imports

In [5]:
import pandas as pd
import numpy as np
import seaborn               as sns
import re

from datetime import datetime

from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot     as plt

from IPython.core.display    import HTML
from IPython.display         import Image

In [2]:
#helper fuction
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


/home/marxcerqueira/.pyenv/versions/3.9.1/envs/ds-ao-dev/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# Data Gathering

## Showcase

In [6]:
# parameters
headers = {'user-agent': 'my-app/0.0.1'}

# URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

#request to URL
page = requests.get(url, headers = headers)

# BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')

# ============== Product Data ===============
#website showcase
products = soup.find('ul', class_= 'products-listing small') #find retorna apenas 1 elemento, pois temos apenas 1 UL, uma vitrine, find_all retorna lista

#list comprehension to get all products id and products category from the first page of 
product_list = products.find_all('article', class_ = 'hm-product-item')

# product id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

# product name
product_list = products.find_all('a', class_ = 'link')
product_name = [p.get_text() for p in product_list]

#price
product_list = soup.find_all('span', class_ = 'price regular')
product_price = [p.get_text() for p in product_list]

data = pd.DataFrame([product_id, product_category, product_name, product_price]).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']

#scrapy time 
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime
0,1008549001,men_jeans_regular,Regular Jeans,$ 19.99,2022-02-14 23:13:21
1,0811993040,men_jeans_regular,Regular Jeans,$ 29.99,2022-02-14 23:13:21
2,1013317006,men_jeans_regular,Hybrid Regular Tapered Joggers,$ 39.99,2022-02-14 23:13:21
3,0875105016,men_jeans_relaxed,Relaxed Jeans,$ 29.99,2022-02-14 23:13:21
4,0875105018,men_jeans_relaxed,Relaxed Jeans,$ 29.99,2022-02-14 23:13:21


In [7]:
data.shape

(36, 5)

## By products

In [94]:
# empty  dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size', 'More sustainable materials']
df_pattern = pd.DataFrame(columns = cols)

for i in range(len(data)):
    # API Request
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'

    print('Product: {}'.format(url))
    
    page = requests.get(url, headers = headers)
    
    # Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # ==================== color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all( 'a', class_='filter-option miniature' )
    
    # color name
    color_name = [p.get( 'data-color' ) for p in product_list]
    
    # product id
    product_id = [p.get( 'data-articlecode' ) for p in product_list]
    
    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range(len(df_color)): #go through all colors and collect each composition 
        # API Request
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'
    
        print('Color: {}'.format(url))

        page = requests.get(url, headers = headers)

        # Beautiful Soup object
        soup = BeautifulSoup(page.text, 'html.parser')
        
        # ============ Product Name =============roduct_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_name = soup.find_all('h1', class_ = 'primary product-item-headline')
#       product_name = [ p.get_text() for p in product_name]
        product_name = product_name[0].get_text()
    
        # ============ Product Price =============
        product_price = soup.find_all('div', class_ = 'primary-row product-item-price')
        product_price = re.findall(r'\d+.?\d+', product_price[0].get_text())[0]     
        
#         df_product_name_price = pd.DataFrame([product_name, product_price]).T
        
        # =================== composition =====================
        product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # create composition dataframe
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna(method = 'ffill')

        # remove pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex = True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex = True)

        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0 )

        #rename columns
        df_composition.columns = ['product_id','composition','fit','product_safety', 'size', 'sustainable_materials']

        #create columns product name and product price
        df_composition['product_name'] = product_name
        df_composition['product_price'] = product_price
        
        #keep new columns if it shows up
        aux = aux + df_composition.columns.tolist() #to guarantee we have all columns of composition unique values

        # merge data color + composition
        df_composition = pd.merge( df_composition, df_color, how = 'left', on = 'product_id')

        # all products
        df_compositions = pd.concat([df_compositions, df_composition], axis = 0)
    
# # Join Showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )

#scrapy time 
df_compositions['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

Product: https://www2.hm.com/en_us/productpage.1008549001.html
Color: https://www2.hm.com/en_us/productpage.1008549001.html
Color: https://www2.hm.com/en_us/productpage.1008549002.html
Color: https://www2.hm.com/en_us/productpage.1008549004.html
Color: https://www2.hm.com/en_us/productpage.1008549006.html
Color: https://www2.hm.com/en_us/productpage.1008549008.html
Product: https://www2.hm.com/en_us/productpage.0811993040.html
Color: https://www2.hm.com/en_us/productpage.0811993040.html
Color: https://www2.hm.com/en_us/productpage.0811993001.html
Color: https://www2.hm.com/en_us/productpage.0811993002.html
Color: https://www2.hm.com/en_us/productpage.0811993003.html
Color: https://www2.hm.com/en_us/productpage.0811993006.html
Color: https://www2.hm.com/en_us/productpage.0811993007.html
Color: https://www2.hm.com/en_us/productpage.0811993021.html
Color: https://www2.hm.com/en_us/productpage.0811993022.html
Color: https://www2.hm.com/en_us/productpage.0811993024.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.0690449040.html
Color: https://www2.hm.com/en_us/productpage.0690449046.html
Color: https://www2.hm.com/en_us/productpage.0690449051.html
Color: https://www2.hm.com/en_us/productpage.0690449056.html
Product: https://www2.hm.com/en_us/productpage.1008549006.html
Color: https://www2.hm.com/en_us/productpage.1008549006.html
Color: https://www2.hm.com/en_us/productpage.1008549001.html
Color: https://www2.hm.com/en_us/productpage.1008549002.html
Color: https://www2.hm.com/en_us/productpage.1008549004.html
Color: https://www2.hm.com/en_us/productpage.1008549008.html
Product: https://www2.hm.com/en_us/productpage.0690449056.html
Color: https://www2.hm.com/en_us/productpage.0690449056.html
Color: https://www2.hm.com/en_us/productpage.0690449001.html
Color: https://www2.hm.com/en_us/productpage.0690449002.html
Color: https://www2.hm.com/en_us/productpage.0690449006.html
Color: https://www2.hm.com/en_us/productpage.0690449007.html
Color: https://www2.

Product: https://www2.hm.com/en_us/productpage.0690449051.html
Color: https://www2.hm.com/en_us/productpage.0690449051.html
Color: https://www2.hm.com/en_us/productpage.0690449001.html
Color: https://www2.hm.com/en_us/productpage.0690449002.html
Color: https://www2.hm.com/en_us/productpage.0690449006.html
Color: https://www2.hm.com/en_us/productpage.0690449007.html
Color: https://www2.hm.com/en_us/productpage.0690449009.html
Color: https://www2.hm.com/en_us/productpage.0690449011.html
Color: https://www2.hm.com/en_us/productpage.0690449013.html
Color: https://www2.hm.com/en_us/productpage.0690449021.html
Color: https://www2.hm.com/en_us/productpage.0690449022.html
Color: https://www2.hm.com/en_us/productpage.0690449024.html
Color: https://www2.hm.com/en_us/productpage.0690449028.html
Color: https://www2.hm.com/en_us/productpage.0690449035.html
Color: https://www2.hm.com/en_us/productpage.0690449036.html
Color: https://www2.hm.com/en_us/productpage.0690449040.html
Color: https://www2.hm

In [213]:
df_compositions['composition'].unique()

array(['Cotton 98%, Spandex 2%', 'Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%', 'Polyester 63%, Cotton 37%',
       'Cotton 100%', 'Cotton 79%, Polyester 20%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 19%, Spandex 2%',
       'Cotton 80%, Polyester 19%, Spandex 1%', 'Polyester 100%',
       'Cotton 93%, Polyester 6%, Spandex 1%',
       'Cotton 91%, Polyester 7%, Spandex 2%',
       'Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%'], dtype=object)

In [ ]:
df_product_category = data[['product_id', 'product_category']].drop_duplicates()
df_product_category

In [46]:
df_compositions.head()

,product_id,composition,fit,product_safety,size,sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,1008549001,"Cotton 98%, Spandex 2%",Regular fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Regular Jeans,19.99,Light denim blue,1008549,001,2022-02-14 23:36:13
1,1008549001,"Polyester 65%, Cotton 35%",Regular fit,NaN,NaN,Recycled cotton 20%,\n\t\t\t\t\t\t\t Regular Jeans,19.99,Light denim blue,1008549,001,2022-02-14 23:36:13
0,1008549002,"Polyester 65%, Cotton 35%",Regular fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Regular Jeans,19.99,Denim blue,1008549,002,2022-02-14 23:36:13
1,1008549002,"Cotton 99%, Spandex 1%",Regular fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Regular Jeans,19.99,Denim blue,1008549,002,2022-02-14 23:36:13
0,1008549004,"Cotton 99%, Spandex 1%",Regular fit,NaN,"The model is 180cm/5'11"" and wears a size 31/32",Recycled cotton 20%,\n\t\t\t\t\t\t\t Regular Jeans,19.99,Dark blue,1008549,004,2022-02-14 23:36:13


In [18]:
df_compositions.columns

Index(['product_id', 'composition', 'fit', 'product_safety', 'size',
       'sustainable_materials', 'pŕoduct_name', 'product_price', 'color_name',
       'style_id', 'color_id', 'scrapy_datetime'],
      dtype='object')

In [208]:
df_data.shape

(99, 16)

In [210]:
len(df_compositions['product_id'].unique())

96

In [48]:
df_data.head()

,product_id,composition,fit,product_safety,size,sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime
0,1008549001,"Cotton 98%, Spandex 2%",Regular fit,NaN,NaN,Recycled cotton 20%,regular jeans,19.99,Light denim blue,1008549,001,2022-02-14 23:36:13
1,1008549001,"Polyester 65%, Cotton 35%",Regular fit,NaN,NaN,Recycled cotton 20%,regular jeans,19.99,Light denim blue,1008549,001,2022-02-14 23:36:13
0,1008549002,"Polyester 65%, Cotton 35%",Regular fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Recycled cotton 20%,regular jeans,19.99,Denim blue,1008549,002,2022-02-14 23:36:13
1,1008549002,"Cotton 99%, Spandex 1%",Regular fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Recycled cotton 20%,regular jeans,19.99,Denim blue,1008549,002,2022-02-14 23:36:13
0,1008549004,"Cotton 99%, Spandex 1%",Regular fit,NaN,"The model is 180cm/5'11"" and wears a size 31/32",Recycled cotton 20%,regular jeans,19.99,Dark blue,1008549,004,2022-02-14 23:36:13


In [211]:
len(df_data['product_id'].unique())

96

In [96]:
df_data['fit'].unique()

array(['regular_fit', 'relaxed_fit', 'loose_fit', 'slim_fit',
       'skinny_fit'], dtype=object)

In [54]:
df_data['size'].unique()

array([nan, 'The model is 185cm/6\'1" and wears a size 31/32',
       'The model is 180cm/5\'11" and wears a size 31/32',
       'The model is 188cm/6\'2" and wears a size 31/30',
       'The model is 187cm/6\'2" and wears a size L',
       'The model is 188cm/6\'2" and wears a size M',
       'The model is 187cm/6\'2" and wears a size M',
       'The model is 187cm/6\'2" and wears a size 32/32',
       'The model is 187cm/6\'2" and wears a size 31/32',
       'The model is 188cm/6\'2" and wears a size 32/30',
       'The model is 189cm/6\'2" and wears a size 31/32',
       'The model is 189cm/6\'2" and wears a size 32/32',
       'The model is 188cm/6\'2" and wears a size 31/32',
       'The model is 182cm/6\'0" and wears a size 33/32',
       'The model is 182cm/6\'0" and wears a size 31',
       'The model is 187cm/6\'2" and wears a size 31',
       'The model is 185cm/6\'1" and wears a size 31',
       'The model is 184cm/6\'0" and wears a size 31/32'], dtype=object)

In [55]:
df_data.isnull().sum()

product_id                 0
composition                0
fit                        0
product_safety           478
size                     229
sustainable_materials    334
product_name               0
product_price              0
color_name                 0
style_id                   0
color_id                   0
scrapy_datetime            0
dtype: int64

In [78]:
df_data.head()

,product_id,composition,fit,product_safety,size,sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model
0,1008549001,"Cotton 98%, Spandex 2%",regular_fit,NaN,NaN,recycled_cotton_20%,regular_jeans,19.99,light_denim_blue,1008549,001,2022-02-14 23:36:13,NaN,NaN
1,1008549001,"Polyester 65%, Cotton 35%",regular_fit,NaN,NaN,recycled_cotton_20%,regular_jeans,19.99,light_denim_blue,1008549,001,2022-02-14 23:36:13,NaN,NaN
0,1008549002,"Polyester 65%, Cotton 35%",regular_fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",recycled_cotton_20%,regular_jeans,19.99,denim_blue,1008549,002,2022-02-14 23:36:13,185,31/32
1,1008549002,"Cotton 99%, Spandex 1%",regular_fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",recycled_cotton_20%,regular_jeans,19.99,denim_blue,1008549,002,2022-02-14 23:36:13,185,31/32
0,1008549004,"Cotton 99%, Spandex 1%",regular_fit,NaN,"The model is 180cm/5'11"" and wears a size 31/32",recycled_cotton_20%,regular_jeans,19.99,dark_blue,1008549,004,2022-02-14 23:36:13,180,31/32


In [136]:
df_data['composition'].unique()

array(['Cotton 98%, Spandex 2%', 'Polyester 65%, Cotton 35%',
       'Cotton 99%, Spandex 1%', 'Polyester 63%, Cotton 37%',
       'Cotton 100%', 'Cotton 79%, Polyester 20%, Spandex 1%',
       'Cotton 77%, Polyester 21%, Spandex 2%',
       'Cotton 78%, Polyester 21%, Spandex 1%',
       'Cotton 79%, Polyester 19%, Spandex 2%',
       'Cotton 80%, Polyester 19%, Spandex 1%', 'Polyester 100%',
       'Cotton 93%, Polyester 6%, Spandex 1%',
       'Cotton 91%, Polyester 7%, Spandex 2%',
       'Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%'], dtype=object)

## Data Cleaning

In [204]:
#product id
df_data = df_compositions.dropna(subset = ['product_id'])

#product name

df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('\n', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('\t', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace('  ', ''))
df_data['product_name'] = df_data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

#product price
df_data['product_price'] = df_data['product_price'].astype(float)

#color name
df_data['color_name'] = df_data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

# #fit
df_data['fit'] = df_data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# #====  size  ======
# #size number
df_data['size_number'] = df_data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
df_data['size_number'] = df_data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x) #group(0) locates the whole match expression

# #size model 
df_data['size_model'] = df_data['size'].str.extract('(\d+/\\d+)') #.str to vectorize the lines, .extracts cant be applied in the whole column

# # #product safety

# =================== sustainable materials ============
df_data['sustainable_materials'] = df_data['sustainable_materials'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# df2 = df_data['sustainable_materials'].str.split()

#recycled cotton / recycled polyester
# creating an empty dataframe as reference to organize the wnanted columns
# thenconcatenete with the main dataframe, but it has to have the same lenght as 'data' dataframe

df2 = df_data[['sustainable_materials']].reset_index(drop=True)
df_ref2 = pd.DataFrame(index = np.arange(len(df_data)), columns = ['recycled_cotton', 'recycled_polyester'])

# ------------ recycled cotton ------------
df_recycled_cotton = df2.loc[df2['sustainable_materials'].str.contains('recycled_cotton', na = True), 'sustainable_materials']
df_recycled_cotton.name = 'recycled_cotton'

df_ref2 = pd.concat([df_ref2, df_recycled_cotton], axis = 1)
df_ref2 = df_ref2.iloc[:, ~df_ref2.columns.duplicated(keep='last')]

# ------------recycled polyester -------------
df_recycled_polyester = df2.loc[df2['sustainable_materials'].str.contains('recycled_polyester', na = True), 'sustainable_materials']
df_recycled_polyester.name = 'recycled_polyester'

df_ref2 = pd.concat([df_ref2, df_recycled_polyester], axis = 1)
df_ref2 = df_ref2.iloc[:, ~df_ref2.columns.duplicated(keep='last')]

# ====================  composition =====================
#break composition by comma
df1 = df_data['composition'].str.split(',', expand = True).reset_index(drop=True)

# cotton / polyester / spandex / 
# creating empty dataframe as reference to organize the wanted columns and 
# then concatanete with the main dataframe, but it has to have the same lenght as 'data' dataframe

df_ref = pd.DataFrame(index = np.arange(len(df_data)), columns = ['cotton', 'polyester', 'spandex'])

# --------------- cotton ----------------
df_cotton_0 = df1.loc[df1[0].str.contains('Cotton', na = True), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains('Cotton', na = True), 1]
df_cotton_1.name = 'cotton'

#combine
df_cotton = df_cotton_0.combine_first(df_cotton_1)

df_ref = pd.concat([df_ref, df_cotton], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep = 'last')] #exclue colunas duplicadas, mantem as diferentes
# df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# -------------- polyester ----------------
df_polyester_0 = df1.loc[df1[0].str.contains('Polyester', na = True), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains('Polyester', na = True), 1]
df_polyester_1.name = 'polyester'

# combine
df_polyester = df_polyester_0.combine_first(df_polyester_1)

# add to reference dataframe
df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep = 'last')] #delet duplicated columns and keep the different ones

# -------------- spandex ------------------
df_spandex_1 = df1.loc[df1[1].str.contains('Spandex', na = True), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains('Spandex', na = True), 2]
df_spandex_2.name = 'spandex'

df_spandex_3 = df1.loc[df1[3].str.contains('Spandex', na = True), 3]
df_spandex_3.name = 'spandex'

# combine
df_spandex_aux = df_spandex_1.combine_first(df_spandex_2)
df_spandex = df_spandex_aux.combine_first(df_spandex_3)

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# # ------------- elasterell -----------
# df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na = True), 1]
# df_elasterell.name = 'elasterell'

# df_ref = pd.concat([df_ref, df_elasterell], axis = 1)
# df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# combine the two dataframe reference
df_ref_final = pd.concat([df_ref, df_ref2], axis = 1)

# add product_id in the aux dataframe
df_aux = pd.concat([df_data['product_id'].reset_index(drop = True), df_ref_final], axis = 1)

# formatt composition data
df_aux['cotton'] = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['polyester'] = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['spandex'] = df_aux['spandex'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['recycled_cotton'] = df_aux['recycled_cotton'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)
df_aux['recycled_polyester'] = df_aux['recycled_polyester'].apply(lambda x: int(re.search('\d+', x).group(0))/100 if pd.notnull(x) else x)

# final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna( 0 ) #choose the highest value between lines
df_data = pd.merge(df_data, df_aux, on = 'product_id', how = 'left')

#d rop columns
df_data = df_data.drop(columns = ['size', 'product_safety','composition'], axis = 1)

#drop duplicates

df_data = df_data.drop_duplicates()

df_data.shape

(99, 16)

In [207]:
df_data.head()

,product_id,fit,sustainable_materials,product_name,product_price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex,recycled_cotton,recycled_polyester
0,1008549001,regular_fit,recycled_cotton_20%,regular_jeans,19.99,light_denim_blue,1008549,001,2022-02-21 10:55:32,NaN,NaN,0.98,0.65,0.02,0.2,0.0
2,1008549002,regular_fit,recycled_cotton_20%,regular_jeans,19.99,denim_blue,1008549,002,2022-02-21 10:55:32,185,31/32,0.99,0.65,0.01,0.2,0.0
4,1008549004,regular_fit,recycled_cotton_20%,regular_jeans,19.99,dark_blue,1008549,004,2022-02-21 10:55:32,180,31/32,0.99,0.63,0.01,0.2,0.0
6,1008549006,regular_fit,recycled_cotton_20%,regular_jeans,19.99,black,1008549,006,2022-02-21 10:55:32,188,31/30,1.00,0.00,0.01,0.2,0.0
8,1008549008,regular_fit,NaN,regular_jeans,19.99,light_denim_blue,1008549,008,2022-02-21 10:55:32,NaN,NaN,0.99,0.65,0.01,0.0,0.0


In [194]:
df_ref_final.tail(11)

,cotton,polyester,spandex,recycled_cotton,recycled_polyester
499,Cotton 78%,Polyester 21%,Spandex 1%,recycled_cotton_20%,NaN
500,Cotton 79%,Polyester 19%,Spandex 2%,recycled_cotton_20%,NaN
501,Cotton 79%,Polyester 19%,Spandex 2%,NaN,recycled_polyester_19%
502,Cotton 79%,Polyester 20%,Spandex 1%,NaN,NaN
503,Cotton 80%,Polyester 19%,Spandex 1%,NaN,NaN
504,Cotton 35%,Polyester 65%,None,recycled_cotton_20%,NaN
505,Cotton 99%,NaN,Spandex 1%,recycled_cotton_20%,NaN
506,Cotton 99%,NaN,Spandex 1%,recycled_cotton_20%,NaN
507,Cotton 99%,NaN,Spandex 1%,recycled_cotton_20%,NaN
508,Cotton 35%,Polyester 65%,None,NaN,NaN


In [198]:
df_aux.head() #choose the highest value 

,product_id,cotton,polyester,spandex,recycled_cotton,recycled_polyester
0,1008549001,0.98,NaN,0.02,0.2,NaN
1,1008549001,0.35,0.65,NaN,0.2,NaN
2,1008549002,0.35,0.65,NaN,0.2,NaN
3,1008549002,0.99,NaN,0.01,0.2,NaN
4,1008549004,0.99,NaN,0.01,0.2,NaN


In [200]:
df_aux.head()

,product_id,cotton,polyester,spandex,recycled_cotton,recycled_polyester
0,0427159001,0.91,0.07,0.02,0.0,0.0
1,0427159002,0.93,0.06,0.01,0.0,0.0
2,0427159003,0.93,0.06,0.01,0.0,0.0
3,0427159004,0.99,0.00,0.01,0.0,0.0
4,0427159005,0.72,0.20,0.01,0.0,0.0


In [189]:
df_ref.head(11)

,cotton,polyester,spandex
0,Cotton 98%,NaN,Spandex 2%
1,Cotton 35%,Polyester 65%,None
2,Cotton 35%,Polyester 65%,None
3,Cotton 99%,NaN,Spandex 1%
4,Cotton 99%,NaN,Spandex 1%
5,Cotton 37%,Polyester 63%,None
6,Cotton 100%,None,None
7,Cotton 99%,NaN,Spandex 1%
8,Cotton 35%,Polyester 65%,None
9,Cotton 99%,NaN,Spandex 1%


In [191]:
df_ref2.tail(20)

,recycled_cotton,recycled_polyester
490,NaN,NaN
491,NaN,NaN
492,NaN,NaN
493,NaN,NaN
494,NaN,NaN
495,NaN,recycled_polyester_21%
496,recycled_cotton_20%,NaN
497,recycled_cotton_19%,NaN
498,NaN,recycled_polyester_21%
499,recycled_cotton_20%,NaN


In [178]:
df1.head(11)

,0,1,2,3
0,Cotton 98%,Spandex 2%,None,None
1,Polyester 65%,Cotton 35%,None,None
2,Polyester 65%,Cotton 35%,None,None
3,Cotton 99%,Spandex 1%,None,None
4,Cotton 99%,Spandex 1%,None,None
5,Polyester 63%,Cotton 37%,None,None
6,Cotton 100%,None,None,None
7,Cotton 99%,Spandex 1%,None,None
8,Polyester 65%,Cotton 35%,None,None
9,Cotton 99%,Spandex 1%,None,None


In [138]:
df_data['sustainable_materials'].unique()

array(['recycled_cotton_20%', nan, 'recycled_polyester_21%',
       'recycled_cotton_19%', 'recycled_polyester_19%'], dtype=object)

In [141]:
df2 = df_data[['sustainable_materials']].reset_index(drop=True)

In [179]:
df2.tail(20)

,sustainable_materials
490,NaN
491,NaN
492,NaN
493,NaN
494,NaN
495,recycled_polyester_21%
496,recycled_cotton_20%
497,recycled_cotton_19%
498,recycled_polyester_21%
499,recycled_cotton_20%


In [170]:
df_recycled_cotton.tail(20)

487                    NaN
488                    NaN
489                    NaN
490                    NaN
491                    NaN
492                    NaN
493                    NaN
494                    NaN
496    recycled_cotton_20%
497    recycled_cotton_19%
499    recycled_cotton_20%
500    recycled_cotton_20%
502                    NaN
503                    NaN
504    recycled_cotton_20%
505    recycled_cotton_20%
506    recycled_cotton_20%
507    recycled_cotton_20%
508                    NaN
509                    NaN
Name: sustainable_materials, dtype: object

In [124]:
df_polyester_1.shape

(137,)

In [116]:
df_polyester_1.head(10)

6               None
27              None
28              None
42     Polyester 20%
43     Polyester 21%
44     Polyester 21%
45     Polyester 21%
46     Polyester 21%
47     Polyester 21%
48     Polyester 19%
Name: polyester, dtype: object

In [132]:
df1[0].unique()

array(['Cotton 98%', 'Polyester 65%', 'Cotton 99%', 'Polyester 63%',
       'Cotton 100%', 'Cotton 79%', 'Cotton 77%', 'Cotton 78%',
       'Cotton 80%', 'Polyester 100%', 'Cotton 93%', 'Cotton 91%',
       'Cotton 72%'], dtype=object)

In [133]:
df1[1].unique()

array([' Spandex 2%', ' Cotton 35%', ' Spandex 1%', ' Cotton 37%', None,
       ' Polyester 20%', ' Polyester 21%', ' Polyester 19%',
       ' Polyester 6%', ' Polyester 7%'], dtype=object)

In [134]:
df1[2].unique()

array([None, ' Spandex 1%', ' Spandex 2%', ' Modal 7%'], dtype=object)

In [135]:
df1[3].unique()

array([None, ' Spandex 1%'], dtype=object)

## Data Insert

In [ ]:
import sqlite3
from sqlalchemy import create_engine

In [ ]:
data_insert = df_data[['']]

In [ ]:
query_showroom_schema = """"
    CREATE TABLE VITRINE (
    
    
    
    )

"""


In [ ]:
#create table
sqlite3.connect('database_hm.sqlite')
conn.execute( query_showroom_schema )
conn.commit()

In [ ]:
# create database connection
create_engine( 'sqlite:///database_hm.sqlite', echo = False)

# data insert
data.insert.to_sql('vitrine', con = conn, if_exists = 'append', index = False )